In [0]:
from google.colab import drive
drive.mount('/content/drive')

##Import libraries and set parameters

In [0]:
from PIL import Image, ImageColor
import requests
from io import BytesIO
import pandas as pd
import json

csvfile = "export-2020-05-04T11_15_24.214Z.csv"
original_imgfolder = "drive/My Drive/Colab data/imgs/"
label_imgfolder = "drive/My Drive/Colab data/imgs_labels/"

## Util function

In [0]:
def draw_single_object(px_img, url, color):
    color = ImageColor.getrgb(color)
    response = requests.get(url)
    img_l = Image.open(BytesIO(response.content))
    px_l = img_l.load()
    for x in range(0,img_l.size[0]):
        for y in range(0,img_l.size[1]):
            if px_l[x,y] == (255,255,255,255):
                px_img[x,y]  = color

## Load csv and make color dictionary

In [5]:
df = pd.read_csv(csvfile)
original_list = []
label_list = []
colors = {}

for i in df['Labeled Data']:
    original_list.append(i)

for i in df['Label']:
    label_list.append(json.loads(i))

print("totol images:", len(label_list))
#make color dictionary
for k in range(len(label_list)):
    for i in range(len(label_list[k]['objects'])):
        if label_list[k]['objects'][i]['value'] not in colors:
            colors[label_list[k]['objects'][i]['value']] = ImageColor.getrgb(label_list[k]['objects'][i]['color'])

print(colors)

totol images: 3
{'ten': (255, 128, 0), 'background': (0, 255, 255), 'queen': (212, 255, 0), 'king': (43, 255, 0), 'ace': (0, 255, 170)}


## Generate label images

In [6]:
for k in range(len(label_list)):
    #make a new blank image with the same size of kth image
    response = requests.get(label_list[k]['objects'][0]['instanceURI'])
    img = Image.open(BytesIO(response.content))
    img = Image.new("RGBA", img.size, color=0)
    px = img.load()
    #color the object in the image
    print("Generating label image " + str(k))
    for i in range(len(label_list[k]['objects'])):
        draw_single_object(px, label_list[k]['objects'][i]['instanceURI'], label_list[k]['objects'][i]['color'])
    
    img.save(label_imgfolder + str(k) + "_L.png")

print("completed!")

Generating label image 0
Generating label image 1
Generating label image 2
completed!


## Save original images

In [7]:
n = 0

for url in original_list:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    print("Saving original image " + str(n))
    img.save(original_imgfolder + str(n) + ".png")
    n = n + 1

print("completed!")

Saving original image 0
Saving original image 1
Saving original image 2
completed!


References:  
1. https://app.labelbox.com/projects  
2. https://medium.com/anolytics/how-to-label-data-for-semantic-segmentation-deep-learning-models-907a996f95f7  